In [1]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import argparse
import sys

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

np.random.seed(1)
print ("cargo")

/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


cargo


In [2]:
def datos():
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)
    print ("Dimensiones train")
    print (mnist.train.images.shape)
    print ("Dimensiones test")
    print (mnist.test.images.shape)
    print ("cargo")
    X_train_o = mnist.train.images[:55000,:]
    Y_train_o = mnist.train.labels[:55000,:]
    X_test_o = mnist.test.images[:10000,:]
    Y_test_o = mnist.test.labels[:10000,:]
    print ("Dimensiones X train")
    print (X_train_o.shape)
    print ("Dimensiones Y train")
    print (Y_train_o.shape)
    print ("Las dimensiones de X son Nx por M, donde Nx son las caracteristicas y M es el numero, por eso hay que trasponerlo")
    X_train=X_train_o.T
    Y_train=Y_train_o.T
    print ("........")
    print ("X train transpuesto", str(X_train.shape))
    print ("Y train transpuesto", str(Y_train.shape))
    X_test=X_test_o.T
    Y_test=Y_test_o.T
    print ("X test transpuesto", str(X_test.shape))
    print ("Y test transpuesto", str(Y_test.shape))
    
    return X_train, Y_train, X_test, Y_test

In [3]:
def create_placeholders(n_x, n_y):
    ### START CODE HERE ### (approx. 2 lines)
    X = tf.placeholder(tf.float32,shape=(n_x, None),name="X")
    Y = tf.placeholder(tf.float32,shape=(n_y, None),name="Y")
    ### Otras variables ###
    
     # variable learning rate
    lr = tf.placeholder(tf.float32)
    # train/test selector for batch normalisation
    tst = tf.placeholder(tf.bool)
    # training iteration
    iter = tf.placeholder(tf.int32)   
    # training iteration
    pkeep = tf.placeholder(tf.float32)      
    return X, Y, tst, iter,pkeep

In [4]:
def initialize_parameters_1(n_x,n_y,tipo=1):
    
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 6 lines of code)
    if(tipo==1):
        W1 =  tf.get_variable("W1", [25,n_x], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
        W2 =  tf.get_variable("W2", [12,25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
        W3 =  tf.get_variable("W3", [n_y,12], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b3 = tf.get_variable("b3", [n_y,1], initializer = tf.zeros_initializer())
    elif(tipo==2):
       
        W1 =  tf.Variable(np.random.randn(25, n_x)*0.01,dtype=tf.float32,name="W1")
        b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
        W2 =  tf.Variable(np.random.randn(12, 25)*0.01,dtype=tf.float32,name="W2")
        b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
        W3 =  tf.Variable(np.random.randn(n_y, 12)*0.01,dtype=tf.float32,name="W3")
        b3 = tf.get_variable("b3", [n_y,1], initializer = tf.zeros_initializer())       
        
        
        
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [5]:
def initialize_parameters(layer,tipo=1):
    parameters={}
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
    L=len(layer)
    for i in range(1, L):
        if(tipo==1):
            parameters['W' + str(i)] =tf.get_variable('W' + str(i), [layer[i], layer[i-1]], initializer = tf.contrib.layers.xavier_initializer(seed = 1))        
            parameters['b' + str(i)] =tf.get_variable('b' + str(i), [layer[i], 1], initializer = tf.zeros_initializer())
        
        if(tipo==2):
            parameters['W' + str(i)] =tf.Variable(np.random.randn(layer[i], layer[i-1])*0.01,dtype=tf.float32,name='W' + str(i))        
            parameters['b' + str(i)] =tf.get_variable('b' + str(i), [layer[i], 1], initializer = tf.zeros_initializer())
            
    
    return parameters

In [6]:
def batchnorm(Ylogits, is_test,iter):
    Ylogits = tf.transpose(Ylogits)
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999,iter) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    
    mean, variance = tf.nn.moments(Ylogits, [0])
    
    update_moving_averages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, None, None, bnepsilon)
    Ybn = tf.transpose(Ybn)
    return Ybn, update_moving_averages

In [7]:
def forward_propagation_1(X, parameters,tst,iter,pkeep):   
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    ### START CODE HERE ### (approx. 5 lines)              # Numpy Equivalents:
    #Z1 = tf.add(tf.matmul(W1,X),b1)  
    Z1 = tf.transpose(tf.matmul(W1,X))                                              # Z1 = np.dot(W1, X) + b1
    Z1B, update_ema1 = batchnorm(Z1, tst,iter)  
    A1 = tf.nn.relu(Z1B)                                              # A1 = relu(Z1)
    A1d = tf.nn.dropout(A1, pkeep)
    A1d=tf.transpose(A1d)
    #Z2 = tf.add(tf.matmul(W2,A1),b2)
    
    Z2 = tf.transpose(tf.matmul(W2,A1d))                                               # Z2 = np.dot(W2, a1) + b2
    Z2B, update_ema2 = batchnorm(Z2, tst,iter)    
    A2 = tf.nn.relu(Z2B)                                              # A2 = relu(Z2)
    A2d = tf.nn.dropout(A2, pkeep)
    A2d=tf.transpose(A2d)
    
    Z3 = tf.add(tf.matmul(W3,A2d),b3)                                             # Z3 = np.dot(W3,Z2) + b3
    ### END CODE HERE ###
    
    update_ema = tf.group(update_ema1,update_ema2)
    return Z3,update_ema

In [8]:
def forward_propagation(X, parameters,tst,iter,pkeep):   
    maximo= (len(parameters))
    cont =1
    l_update_ema=[]
    for i in range(1,maximo+1,2):
        if (cont==((maximo)/2)):
            break
        if(i==1):
            Z=linear_forward(X,parameters['W'+str(cont)],parameters['b'+str(cont)])
            #Z = tf.matmul(parameters['W'+str(cont)],X)
            ZB, update_ema = batchnorm(Z, tst,iter)
            A = tf.nn.relu(ZB)                                              
            #Ad = sess.run(tf.nn.dropout(A, pkeep))
            Ad = tf.nn.dropout(A, pkeep)
        else:
            #Z = tf.matmul(parameters['W'+str(cont)],Ad)
            Z=linear_forward(Ad,parameters['W'+str(cont)],parameters['b'+str(cont)])
            ZB, update_ema = batchnorm(Z, tst,iter)
            A = tf.nn.relu(ZB)                                        
            #Ad = tf.nn.dropout(A, pkeep)
            Ad = tf.nn.dropout(A, pkeep)
        l_update_ema.append(update_ema)
        cont=cont+1
            
    #Z3 = tf.add(tf.matmul(parameters['W'+str(cont)],Ad),parameters['b'+str(cont)])    
    Z3=linear_forward(Ad,parameters['W'+str(cont)],parameters['b'+str(cont)])
    if((cont==3)):
        print ("entra")
        update_ema = tf.group(l_update_ema[0],l_update_ema[1])
    if((cont==4)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2])
    if((cont==5)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],update_ema[3])  
    if((cont==6)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],update_ema[3],update_ema[4])      
    return Z3

In [9]:
def linear_forward(A, W, b):
    Z = tf.add(tf.matmul(W,A),b)  
    return Z

In [10]:
# GRADED FUNCTION: compute_cost 
def compute_cost(Z3, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    ### END CODE HERE ###
    
    return cost

In [11]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [16]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 30, minibatch_size = 32, print_cost = False,pkeepv=1.0):

    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    cost_train = []                                        # To keep track of the cost
    cost_test = []
    layer=[n_x,25,12,n_y]
    #print (n_x)
    #print (m)
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y,tst,iter,pkeep= create_placeholders(n_x, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    ### inicializar los parametros y la arquitectura de la red
    #parameters = initialize_parameters(n_x,n_y,2)
    parameters = initialize_parameters(layer,1)
    #print (parameters)
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = forward_propagation(X, parameters,tst,iter,pkeep)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3,Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    
    ##### Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        i=0
        for epoch in range(num_epochs):
            i=i+1
            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            #print ("mini "+str(num_minibatches))
            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                #_ , minibatch_cost, _ = sess.run([optimizer, cost, update_ema], feed_dict={X: minibatch_X, Y: minibatch_Y, tst: False, iter: i,pkeep: pkeepv})
                ### END CODE HERE ###
                #minibatch_cost = sess.run(cost, feed_dict={X: minibatch_X, Y: minibatch_Y, tst: False})
                sess.run(optimizer, {X: minibatch_X, Y: minibatch_Y, tst: False, iter: i})
                #sess.run(update_ema, {X: minibatch_X, Y: minibatch_Y, tst: False, iter: i})
                
                epoch_cost += minibatch_cost / num_minibatches
                #if(i==30):
                #    break

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                cost_train.append(epoch_cost)
                c_test = sess.run([ cost], feed_dict={X: minibatch_X, Y: minibatch_Y, tst: True,pkeep: 1.0})
                cost_test.append(c_test)
  
        if(print_cost== True):
            # plot the cost
            plt.figure("Figura1")
            plt.plot(np.squeeze(cost_train))
            plt.ylabel('cost')
            plt.xlabel('iterations (per tens)')
            plt.title("Train Learning rate =" + str(learning_rate))

            plt.figure("Figura2")
            plt.plot(np.squeeze(cost_test))
            plt.ylabel('cost')
            plt.xlabel('iterations (per tens)')
            plt.title("Test Learning rate =" + str(learning_rate))        


            plt.figure("Figura3")
            plt.plot(np.squeeze(cost_train))
            plt.plot(np.squeeze(cost_test))
            plt.ylabel('cost')
            plt.xlabel('iterations (per tens)')
            plt.title("Test Learning rate =" + str(learning_rate)) 
            plt.legend(loc=4)
            plt.show()
        
        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions, xq no recibe a3. Claro no lo tengo. ¿como sabe que debe calcularlo?
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        acc_train=sess.run(accuracy, feed_dict ={X: X_train, Y: Y_train,tst: False,pkeep: 1.0})
        acc_test=sess.run(accuracy ,feed_dict ={X: X_test, Y: Y_test,tst: True,pkeep: 1.0})
        
        #print ("Train Accuracy :", accuracy.eval({X: X_train, Y: Y_train,tst: False,pkeep: 1.0}))
        #print ("Test Accuracy: ", accuracy.eval({X: X_test, Y: Y_test,tst: True,pkeep: 1.0}))
        '''
        print ("Train Accuracy:", sess.run(accuracy, feed_dict ={X: X_train, Y: Y_train})

        print ("Test Accuracy:", sess.run(accuracy, feed_dict ={X: X_test, Y: Y_test})
        
        '''
        print ("-----Termino------")
        return parameters,acc_train,acc_test

In [17]:
X_train, Y_train, X_test, Y_test = datos()

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Dimensiones train
(55000, 784)
Dimensiones test
(10000, 784)
cargo
Dimensiones X train
(55000, 784)
Dimensiones Y train
(55000, 10)
Las dimensiones de X son Nx por M, donde Nx son las caracteristicas y M es el numero, por eso hay que trasponerlo
........
X train transpuesto (784, 55000)
Y train transpuesto (10, 55000)
X test transpuesto (784, 10000)
Y test transpuesto (10, 10000)


In [18]:
%%time
#a_learning_rate = [0.0001, 0.009, 0.005, 0.001, 0.09, 0.05, 0.01, 0.9, 0.5, 0.1]
#a_pkeepv = [1,0.95,0.90,0.85,0.80]
a_learning_rate = [0.0001]
a_pkeepv = [0.90]
nume=30
result=[]
for e_learning_rate in a_learning_rate:
    for e_pkeepv in a_pkeepv:
        print (e_learning_rate , e_pkeepv)
        parameters,acc_train,acc_test = model(X_train, Y_train, X_test, Y_test,learning_rate=e_learning_rate,pkeepv=e_pkeepv,num_epochs=nume,print_cost=False)
        result.append([e_learning_rate,e_pkeepv,acc_train,acc_test])

0.0001 0.9
entra


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_3' with dtype float
	 [[Node: Placeholder_3 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_3', defined at:
  File "/root/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/root/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).star

In [148]:
print ("hola")

hola


In [13]:
result

[[0.0001, 0.9, 0.96350908, 0.95249999]]